In [347]:
import pybaseball
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', 1600)
pd.set_option('display.max_columns',1500)
pd.set_option('display.width', 1500)

In [348]:
injuries2 = pd.read_excel("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/Injuries_Full.xlsx")

In [349]:
injuries2['Player'] =injuries2['Player'].str.upper()

## Tommy John Injuries

In [350]:
tj_injuries = pd.read_excel("Tommy John Surgery List (@MLBPlayerAnalys).xlsx", dtype='object')
mlb_injuries = tj_injuries[(tj_injuries['Level'] == 'MLB') & (tj_injuries['Position'] == 'P')]


In [351]:
mlb_injuries['Player'] = mlb_injuries['Player'].str.replace("Mark Leiter Jr.",'Mark Leiter')
mlb_injuries['Player'] = mlb_injuries['Player'].str.replace("Manuel Corpas",'MANNY CORPAS')


/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [352]:
mlb_injuries['Player'] = mlb_injuries['Player'].str.strip()

/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [353]:
mlb_injuries['TJ Surgery Date'] = pd.to_datetime(mlb_injuries['TJ Surgery Date'])

/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [354]:
mlb_injuries['surgery_year'] = [x.year - 1 if
                                x.month < 6 else
                                x.year for x in mlb_injuries['TJ Surgery Date']]

/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [355]:
mlb_injuries['Player'] = mlb_injuries['Player'].str.upper()

/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [356]:
mlb_injuries = mlb_injuries[['Player','surgery_year']]

In [357]:
mlb_injuries['tj'] = 1

In [358]:
def tj_injury(player, season):
    if mlb_injuries[(mlb_injuries['Player'] == player) & 
                   (mlb_injuries['surgery_year'] < season)]['tj'].sum() > 0:
        return 1
    else:
        return 0 

In [359]:
def tj_injury_year(player, season):
    
    temp = mlb_injuries[(mlb_injuries['Player'] == player) & 
                   (mlb_injuries['surgery_year'] < season)]
    
    if temp['tj'].sum() > 0:
        return temp["surgery_year"].max()
    else:
        return 0 

## Grab Pitcher Stats By Year

In [360]:
#pitching_stats = pybaseball.pitching_stats(2000, 2020)
pitching_stats = pd.read_csv("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/pitching_stats_2009_2020.csv")

In [361]:
pitching_stats.to_csv("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/pitching_stats_2009_2020.csv",
                     index=False)

In [362]:
pitch_types = ['FB%', 'SL%', 'CT%', 'CB%', 'SF%', 'KN%']

In [363]:
pitching_stats.sort_values("Season", inplace=True)

In [364]:
pitching_stats['Name'] = pitching_stats['Name'].str.upper()

In [365]:
pitchers = pitching_stats.groupby("Name").agg({"Season":"min"}).reset_index()

pitchers_gt_2001 = pitchers[pitchers['Season'] >= 2001]

In [366]:
pitches_injury = pd.merge(pitching_stats, injuries2,
         left_on =['Name','Season'],
         right_on = ['Player','Season'],
        how='left')

In [367]:
ps = pitches_injury.groupby("Name").agg({"Season":pd.Series.nunique,
                                        "Age":"min"})

In [368]:
ps = ps.rename(columns={"Season":"Num_Seasons",
                       "Age":"first_age"}).reset_index()

In [369]:
pitches_injury = pitches_injury.merge(ps,on='Name')

In [370]:
pitches_injury = pitches_injury[pitches_injury['Num_Seasons'] > 1]
pitches_injury = pitches_injury[pitches_injury['first_age'] < 28]

In [371]:
pitches_injury = pitches_injury[pitches_injury['Season'] >= 2002]

In [372]:
pitches_injury["tj"] = [tj_injury(x,y) for x,y in pitches_injury[['Name','Season']].itertuples(index=False)]

In [373]:
pitches_injury["tj_year"] = [tj_injury_year(x,y) for x,y in pitches_injury[['Name','Season']].itertuples(index=False)]

## Pitching Injuries 2

In [374]:
pitches_injury['Injury'] = pitches_injury['Injury'].str.upper()

In [375]:
pitches_injury['injured'] = [1 if "ELBOW" in x
                             or "SHOULDER" in x
                             or "BACK" in x 
                             or "OBLIQUE" in x
                             or "ARM" in x 
                             or "KNEE" in x
                             or "FINGER" in x
                             or "GROIN" in x
                             or "HIP" in x
                             or "NECK" in x
                             else 0 for x in pitches_injury['Injury'].fillna("")]

In [376]:
# pitches_injury['injured_tag']  = [1 if (season <= injured_year) 
#                              and (injured == 1) and (season > (injured_year - 1))
# else 0 
# for name, season, injured_year, injured 
# in pitches_injury[['Name','Season','surgery_year','injured']].itertuples(index=False)]

In [377]:
pitches_injury = pitches_injury.drop_duplicates(["Name","Season","injured"])

In [378]:
pitches_injury = pitches_injury.sort_values("injured",ascending=False).drop_duplicates(subset=['Name','Season'])

In [379]:
pitch_types = ['FB%', 'SL%', 'CT%', 'CB%', 'CH%', 'SF%']
pitch_vels = ['FBv', 'SLv', 'CTv', 'CBv','CHv', 'SFv']

In [380]:
cols = ["injured",
 "FA-Z (pfx)",
 "FT-Z (pfx)",
 "FC-Z (pfx)",
 "FS-Z (pfx)",
 "FO-Z (pfx)",
 "SI-Z (pfx)",
 "SL-Z (pfx)",
 "CU-Z (pfx)",
 "KC-Z (pfx)",
 "EP-Z (pfx)",
 "CH-Z (pfx)",
 "SC-Z (pfx)",
 "KN-Z (pfx)"]

## Feature Engineering

In [381]:
def number_seasons_prior(Player, Season):
    return Season - pitches_injury[pitches_injury['Name'] == Player]['Season'].min()

In [382]:
pitches_injury['num_seasons_a_priori'] = [number_seasons_prior(x,y) 
                                 for x,y 
                                 in pitches_injury[['Name', 'Season']].itertuples(index=False)]

In [383]:
pitches_injury['seasons_until_injury'] = [x + 1 if y == 1 else 0 for x,y 
                                          in pitches_injury[['num_seasons_a_priori', 'injured']]
                                          .itertuples(index=False)]

In [384]:
for pitch_type in pitch_types:
    pitches_injury[pitch_type[:-1] + "_count"] = pitches_injury[pitch_type] * pitches_injury['Pitches']

In [385]:
def pitch_diff(Player, Season):
    temp_player = pitches_injury[(pitches_injury['Name'] == Player)
                                &(pitches_injury['Season'] <= Season)
                                & (pitches_injury['Season'] > Season - 3)]
    
    curr_player = pitches_injury[(pitches_injury['Name'] == Player)
                                &(pitches_injury['Season'] == Season)]
    
    temp_df = pd.DataFrame(temp_player[delta_types].mean()).T
    temp_df['Player'] = Player
    temp_df['Season'] = Season
    return temp_df

In [386]:
pitches_injury['pitches_per_game'] = pitches_injury['Pitches'] / pitches_injury['G']
pitches_injury["IP_per_G"] = pitches_injury['IP'] / pitches_injury['G']
pitches_injury["P_per_IP"] = pitches_injury['Pitches'] / pitches_injury['IP']


In [387]:
def shift_injury(player, season):
    if pitches_injury[(pitches_injury['Name'] == player) & 
                     (pitches_injury['Season'] == season + 1)]['injured'].sum() > 0 :
        return 1
    else:
        return 0

In [389]:
def prior_injury(player, season):
    if pitches_injury[(pitches_injury['Name'] == player) & 
                     (pitches_injury['Season'] < season)]['injured'].sum() > 0 :
        return 1
    else:
        return 0

In [390]:
def prior_injury_last_2_seasons(player, season):
    if pitches_injury[(pitches_injury['Name'] == player) & 
                     (pitches_injury['Season'] < season) & 
                     (pitches_injury['Season'] >= season - 1)]['injured'].sum() > 0 :
        return 1
    else:
        return 0

In [391]:
pitches_injury['injury_lagged'] = [shift_injury(x,y) for x,y in pitches_injury[['Name','Season']].itertuples(index=False)]

In [392]:
pitches_injury['prior_injury'] = [prior_injury(x,y) for x,y in pitches_injury[['Name','Season']].itertuples(index=False)]

In [393]:
pitches_injury['prior_injury_last_2'] = [prior_injury_last_2_seasons(x,y) for x,y in pitches_injury[['Name','Season']].itertuples(index=False)]

In [395]:
pitches_injury["FA% (pi)"]  =  [x if x == x 
                                else y if y == y
                                else z for x,y,z 
                                in pitches_injury[["FA% (pi)","FB%","FA% (pfx)"]].itertuples(index=False)]

In [396]:
pitches_injury["vFA (pi)"]  =  [x if x == x 
                                else y if y == y
                                else z for x,y,z 
                                in pitches_injury[["vFA (pi)","FBv","vFA (pfx)"]].itertuples(index=False)]

In [397]:
pitches_injury["FC% (pi)"]  =  [x if x == x 
                                else y if y == y
                                else z for x,y,z 
                                in pitches_injury[["FC% (pi)","CT%","FC% (pfx)"]].itertuples(index=False)]

In [398]:
pitches_injury["vFC (pi)"]  =  [x if x == x 
                                else y if y == y
                                else z for x,y,z 
                                in pitches_injury[["vFC (pi)","CTv","vFC (pfx)"]].itertuples(index=False)]

In [399]:
pitches_injury["SI% (pi)"]  =  [x if x == x 
                                else y for x,y 
                                in pitches_injury[["SI% (pi)","SI% (pfx)"]].itertuples(index=False)]

In [400]:
pitches_injury["vSI (pi)"]  =  [x if x == x 
                                else y for x,y 
                                in pitches_injury[["vSI (pi)","vSI (pfx)"]].itertuples(index=False)]

In [401]:
pitches_injury["FA-Z (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["FA-Z (pi)","FA-Z (pfx)"]].itertuples(index=False)]

pitches_injury["FA-X (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["FA-X (pi)","FA-X (pfx)"]].itertuples(index=False)]

pitches_injury["wFA (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["wFA (pi)","wFA (pfx)"]].itertuples(index=False)]

In [402]:
pitches_injury["SI-Z (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["SI-Z (pi)","SI-Z (pfx)"]].itertuples(index=False)]

pitches_injury["SI-X (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["SI-X (pi)","SI-X (pfx)"]].itertuples(index=False)]

pitches_injury["wSI (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["wSI (pi)","wSI (pfx)"]].itertuples(index=False)]

In [403]:
pitches_injury["FC-Z (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["FC-Z (pi)","FC-Z (pfx)"]].itertuples(index=False)]

pitches_injury["FC-X (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["FC-X (pi)","FC-X (pfx)"]].itertuples(index=False)]

pitches_injury["wFC (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["wFC (pi)","wFC (pfx)"]].itertuples(index=False)]

In [404]:
pitch_types = ['CH% (pi)', 'CS% (pi)', 'CU% (pi)', 'FA% (pi)',
               'FC% (pi)', 'FS% (pi)', 'KN% (pi)', 'SB% (pi)',
               'SI% (pi)', 'SL% (pi)', 'XX% (pi)',
              'CH-X (pi)', 'CS-X (pi)', 'CU-X (pi)', 'FA-X (pi)',
               'FC-X (pi)', 'FS-X (pi)', 'KN-X (pi)', 'SB-X (pi)',
               'SI-X (pi)', 'SL-X (pi)', 'XX-X (pi)',
               'CH-Z (pi)', 'CS-Z (pi)', 'CU-Z (pi)', 'FA-Z (pi)',
               'FC-Z (pi)', 'FS-Z (pi)', 'KN-Z (pi)', 'SB-Z (pi)',
               'SI-Z (pi)', 'SL-Z (pi)', 'XX-Z (pi)',
               'wCH (pi)', 'wCS (pi)', 'wCU (pi)',
               'wFA (pi)', 'wFC (pi)', 'wFS (pi)',
               'wKN (pi)', 'wSB (pi)', 'wSI (pi)',
               'wSL (pi)', 'wXX (pi)'
               
              ]

In [405]:
pitches_injury['sinkers_cutters_percent'] = pitches_injury['FC% (pi)'].fillna(0) + pitches_injury['SI% (pi)'].fillna(0)

In [406]:
pitches_injury['other_pitch_types_percent'] = (pitches_injury['CH% (pi)'].fillna(0) +
                                       pitches_injury['CS% (pi)'].fillna(0) +
                                       pitches_injury['CU% (pi)'].fillna(0) +
                                       pitches_injury['FS% (pi)'].fillna(0) +
                                       pitches_injury['KN% (pi)'].fillna(0) +
                                       pitches_injury['SB% (pi)'].fillna(0) +
                                       pitches_injury['SL% (pi)'].fillna(0) +
                                       pitches_injury['XX% (pi)'].fillna(0)
                                      )

In [407]:
pitches_injury['other_pitch_types_w'] = (pitches_injury['wCH (pi)'].fillna(0) +
                                       pitches_injury['wCS (pi)'].fillna(0) +
                                       pitches_injury['wCU (pi)'].fillna(0) +
                                       pitches_injury['wFS (pi)'].fillna(0) +
                                       pitches_injury['wKN (pi)'].fillna(0) +
                                       pitches_injury['wSB (pi)'].fillna(0) +
                                       pitches_injury['wSL (pi)'].fillna(0) +
                                       pitches_injury['wXX (pi)'].fillna(0)
                                      )

In [408]:
pitches_injury['other_pitch_types_X'] = (pitches_injury['CH-X (pi)'].fillna(0) +
                                       pitches_injury['CS-X (pi)'].fillna(0) +
                                       pitches_injury['CU-X (pi)'].fillna(0) +
                                       pitches_injury['FS-X (pi)'].fillna(0) +
                                       pitches_injury['KN-X (pi)'].fillna(0) +
                                       pitches_injury['SB-X (pi)'].fillna(0) +
                                       pitches_injury['SL-X (pi)'].fillna(0) +
                                       pitches_injury['XX-X (pi)'].fillna(0)
                                      )

In [409]:
pitches_injury['other_pitch_types_Z'] = (pitches_injury['CH-Z (pi)'].fillna(0) +
                                       pitches_injury['CS-Z (pi)'].fillna(0) +
                                       pitches_injury['CU-Z (pi)'].fillna(0) +
                                       pitches_injury['FS-Z (pi)'].fillna(0) +
                                       pitches_injury['KN-Z (pi)'].fillna(0) +
                                       pitches_injury['SB-Z (pi)'].fillna(0) +
                                       pitches_injury['SL-Z (pi)'].fillna(0) +
                                       pitches_injury['XX-Z (pi)'].fillna(0)
                                      )

In [410]:
others = ["other_pitch_types_percent","other_pitch_types_w","other_pitch_types_X","other_pitch_types_Z"]

In [411]:
def calc_accel(df, field):
    temp = (df
                   .groupby(['Name','Season'])[field]
                   .sum().groupby(level=0, group_keys=False)
                   .shift(+2).reset_index())

    temp.rename(columns={field:"{}_lag_2".format(field)}, inplace = True)

    if "{}_lag_2".format(field) in df.columns:
        df.drop(columns=["{}_lag_2".format(field)],inplace=True)
    
    df = pd.merge(df, temp, on=['Name','Season'])

    df['{}_diff_2'.format(field)] = df[field] - df["{}_lag_2".format(field)]
    df['{}_accel_2'.format(field)] = df['{}_diff_2'.format(field)] / 3
    return df

In [413]:
pitches_injury['years_since_tj'] = [y - x if x > 0 
                                    else 0 
                                    for x,y 
                                    in pitches_injury[['tj_year',"Season"]].itertuples(index=False)]

In [414]:
cols = ['Pitches','G', "IP", "sinkers_cutters_percent",'vFA (pi)','vFC (pi)',"vSI (pi)","FA% (pi)"]

In [415]:
pitches_injury['SO_G'] = pitches_injury['SO'] / pitches_injury['G']

In [416]:
## Generate acceleration features

In [417]:
for col in cols:
    pitches_injury = calc_accel(pitches_injury, col)

In [418]:
for col in pitch_types:
    pitches_injury = calc_accel(pitches_injury, col)

In [419]:
for col in others:
    pitches_injury = calc_accel(pitches_injury, col)

In [420]:
pitches_injury = calc_accel(pitches_injury, 'SO_G')

In [421]:
pitches_injury = calc_accel(pitches_injury, 'P_per_IP')

In [422]:
pitches_injury = calc_accel(pitches_injury, 'vSL (pi)')

TBF, BB, WP

In [423]:
other_stats = ["walk_rate","wild_pitch_rate"]

In [424]:
pitches_injury['walk_rate'] = ((pitches_injury['BB'] - pitches_injury['IBB']) / pitches_injury['TBF'])

In [425]:
pitches_injury['wild_pitch_rate'] = (pitches_injury['WP'] / pitches_injury['Pitches'])

In [427]:
pitches_injury['win_rec'] = (pitches_injury['W'] / (pitches_injury['L'] + pitches_injury['W'])).fillna(0)

In [428]:
for col in other_stats:
    pitches_injury = calc_accel(pitches_injury, col)

In [471]:
more_stats = ["wFC/C (pi)","wSI/C (pi)","wFA/C (pi)"]

In [472]:
for col in more_stats:
    pitches_injury = calc_accel(pitches_injury, col)

In [429]:
(pitches_injury[pitches_injury['Season'] > 2013]
 .groupby([     
     'tj',
     #pd.cut(pitches_injury[pitches_injury['Season'] > 2013]["Age"], 5, duplicates="drop"),
           pd.cut(pitches_injury[pitches_injury['Season'] > 2013]['wKN (pi)_accel_2'], bins=2,duplicates="drop"),
     
          ])
 .agg({"injury_lagged":['mean','sum','count']}))

injury_lagged           
                             mean  sum count
tj wKN (pi)_accel_2                         
0  (-9.112, -3.083]          0.00  0.0   2.0
   (-3.083, 2.933]           0.25  1.0   4.0
1  (-9.112, -3.083]           NaN  NaN   NaN
   (-3.083, 2.933]           0.00  0.0   2.0

In [430]:
pitch_types = ['CH% (pi)', 'CS% (pi)', 'CU% (pi)', 'FA% (pi)',
               'FC% (pi)', 'FS% (pi)', 'KN% (pi)', 'SB% (pi)',
               'SI% (pi)', 'SL% (pi)', 'XX% (pi)',
              'CH-X (pi)', 'CS-X (pi)', 'CU-X (pi)', 'FA-X (pi)',
               'FC-X (pi)', 'FS-X (pi)', 'KN-X (pi)', 'SB-X (pi)',
               'SI-X (pi)', 'SL-X (pi)', 'XX-X (pi)',
               'CH-Z (pi)', 'CS-Z (pi)', 'CU-Z (pi)', 'FA-Z (pi)',
               'FC-Z (pi)', 'FS-Z (pi)', 'KN-Z (pi)', 'SB-Z (pi)',
               'SI-Z (pi)', 'SL-Z (pi)', 'XX-Z (pi)',
               'wCH (pi)', 'wCS (pi)', 'wCU (pi)',
               'wFA (pi)', 'wFC (pi)', 'wFS (pi)',
               'wKN (pi)', 'wSB (pi)', 'wSI (pi)',
               'wSL (pi)', 'wXX (pi)',
               'CH% (pi)_accel_2', 'CS% (pi)_accel_2', 'CU% (pi)_accel_2', 'FA% (pi)_accel_2',
               'FC% (pi)_accel_2', 'FS% (pi)_accel_2', 'KN% (pi)_accel_2', 'SB% (pi)_accel_2',
               'SI% (pi)_accel_2', 'SL% (pi)_accel_2', 'XX% (pi)_accel_2',
              'CH-X (pi)_accel_2', 'CS-X (pi)_accel_2', 'CU-X (pi)_accel_2', 'FA-X (pi)_accel_2',
               'FC-X (pi)_accel_2', 'FS-X (pi)_accel_2', 'KN-X (pi)_accel_2', 'SB-X (pi)_accel_2',
               'SI-X (pi)_accel_2', 'SL-X (pi)_accel_2', 'XX-X (pi)_accel_2',
               'CH-Z (pi)_accel_2', 'CS-Z (pi)_accel_2', 'CU-Z (pi)_accel_2', 'FA-Z (pi)_accel_2',
               'FC-Z (pi)_accel_2', 'FS-Z (pi)_accel_2', 'KN-Z (pi)_accel_2', 'SB-Z (pi)_accel_2',
               'SI-Z (pi)_accel_2', 'SL-Z (pi)_accel_2', 'XX-Z (pi)_accel_2',
               'wCH (pi)_accel_2', 'wCS (pi)_accel_2', 'wCU (pi)_accel_2',
               'wFA (pi)_accel_2', 'wFC (pi)_accel_2', 'wFS (pi)_accel_2',
               'wKN (pi)_accel_2', 'wSB (pi)_accel_2', 'wSI (pi)_accel_2',
               'wSL (pi)_accel_2', 'wXX (pi)_accel_2'
               
              ]

In [431]:
pitch_types_accel = ['CH% (pi)_accel_2', 'CS% (pi)_accel_2', 'CU% (pi)_accel_2', 'FA% (pi)_accel_2',
               'FC% (pi)_accel_2', 'FS% (pi)_accel_2', 'KN% (pi)_accel_2', 'SB% (pi)_accel_2',
               'SI% (pi)_accel_2', 'SL% (pi)_accel_2', 'XX% (pi)_accel_2',
              'CH-X (pi)_accel_2', 'CS-X (pi)_accel_2', 'CU-X (pi)_accel_2', 'FA-X (pi)_accel_2',
               'FC-X (pi)_accel_2', 'FS-X (pi)_accel_2', 'KN-X (pi)_accel_2', 'SB-X (pi)_accel_2',
               'SI-X (pi)_accel_2', 'SL-X (pi)_accel_2', 'XX-X (pi)_accel_2',
               'CH-Z (pi)_accel_2', 'CS-Z (pi)_accel_2', 'CU-Z (pi)_accel_2', 'FA-Z (pi)_accel_2',
               'FC-Z (pi)_accel_2', 'FS-Z (pi)_accel_2', 'KN-Z (pi)_accel_2', 'SB-Z (pi)_accel_2',
               'SI-Z (pi)_accel_2', 'SL-Z (pi)_accel_2', 'XX-Z (pi)_accel_2',
               'wCH (pi)_accel_2', 'wCS (pi)_accel_2', 'wCU (pi)_accel_2',
               'wFA (pi)_accel_2', 'wFC (pi)_accel_2', 'wFS (pi)_accel_2',
               'wKN (pi)_accel_2', 'wSB (pi)_accel_2', 'wSI (pi)_accel_2',
               'wSL (pi)_accel_2', 'wXX (pi)_accel_2'
              ]

In [432]:
pitch_types = ['CH-X (pi)', 'CS-X (pi)', 'CU-X (pi)', 'FA-X (pi)',
               'FC-X (pi)', 'FS-X (pi)', 'KN-X (pi)', 'SB-X (pi)',
               'SI-X (pi)', 'SL-X (pi)', 'XX-X (pi)']

## World Series?

In [434]:
ws_teams = {"Red Sox":2013,
"Cardinals":2013,
"Giants":2014,
"Royals":2014,
"Royals":2015,
"Mets":2015,
"Cubs":2016,
"Indians":2016,
"Astros":2017,
"Dodgers":2017,
"Red Sox":2018,
"Dodgers":2018,
"Nationals":2019,
"Astros":2019,
}

In [435]:
pitches_injury['ws_pitcher'] = [1 if y in ws_teams 
 and ws_teams[y] == x 
 else 0 
 for x,y in pitches_injury[['Season','Team_x']].itertuples(index=False)]

In [436]:
## Acceleration over a three year period would be the final prop - initial prop / 3. The Acceleration would then be
## the rate at which the proportion increased over the three year period. 

In [437]:
pitches_injury['pitches_per_bat'] = (pitches_injury['Pitches'] / pitches_injury['TBF'])

In [476]:
training_cols = ["Name","Season","injured","injury_lagged", "Age",'Pitches','IP','G',"P_per_IP_accel_2",
                'prior_injury','sinkers_cutters_percent','prior_injury_last_2','P_per_IP',"pitches_per_bat",
                 'Pitches_accel_2','G_accel_2','IP_accel_2',"tj","sinkers_cutters_percent_accel_2","IP_per_G",
                 "pitches_per_game",
                 "other_pitch_types_percent_accel_2","other_pitch_types_w_accel_2","years_since_tj",
                 "other_pitch_types_X_accel_2","other_pitch_types_Z_accel_2","SO_G_accel_2","ws_pitcher",
                 "walk_rate","walk_rate_accel_2","wild_pitch_rate","wild_pitch_rate_accel_2",
                 'FA% (pi)','FC% (pi)','SI% (pi)',
              'FA-X (pi)','FC-X (pi)','SI-X (pi)',
                'FA-Z (pi)','FC-Z (pi)','SI-Z (pi)',
               'wFA (pi)', 'wFC (pi)', 'wSI (pi)',
                'FA% (pi)_accel_2',
               'FC% (pi)_accel_2',
               'SI% (pi)_accel_2',
               'FA-X (pi)_accel_2',
               'FC-X (pi)_accel_2',
               'SI-X (pi)_accel_2',
               'FA-Z (pi)_accel_2',
               'FC-Z (pi)_accel_2','wFC/C (pi)','wFC/C (pi)_accel_2',
                 'wSI/C (pi)','wSI/C (pi)_accel_2','wFA/C (pi)','wFA/C (pi)_accel_2',
               'SI-Z (pi)_accel_2',"win_rec",
               'wFA (pi)_accel_2', 'wSI (pi)_accel_2','wFC (pi)_accel_2',
                 'vFA (pi)','vFC (pi)','vSI (pi)','vFC (pi)_accel_2','vFA (pi)_accel_2','vSI (pi)_accel_2',
                 'SL% (pi)',
                 'SL% (pi)_accel_2',
                 'SL-X (pi)',
                 'SL-X (pi)_accel_2',
                 'SL-Z (pi)',
                 'SL-Z (pi)_accel_2',
                'vSL (pi)',
                 'vSL (pi)_accel_2',
                    'wSL (pi)',
                 'wSL (pi)_accel_2'
    
                ]

In [480]:
pitches_injury_2015[pitches_injury_2015['vFA (pi)'].isnull()][["IP",'vFA (pi)',"FB%","FA% (pi)","Season","Team_x",
                                                              "injury_lagged"]]

,IP,vFA (pi),FB%,FA% (pi),Season,Team_x,injury_lagged
1324,66.2,NaN,NaN,NaN,2016.0,Indians,0


In [483]:
pitches_injury_2013 = pitches_injury[pitches_injury['Season'] > 2013]
pitches_injury_2015 = pitches_injury[(pitches_injury['Season'] > 2013) & (pitches_injury['Season'] < 2018)]
pitches_injury_2019 =  pitches_injury[pitches_injury['Season'] == 2018]
pitches_injury_2020 = pitches_injury[pitches_injury['Season'] == 2019]

In [484]:
pitches_injury_2015 = pitches_injury_2015[(pitches_injury_2015['IP'] > 10)]

In [485]:
## Removing Pitchers who don't have a sufficient history to create acceleration variables. 
pitches_injury_2015 = pitches_injury_2015[~pitches_injury_2015['SO_G_accel_2'].isnull()]

In [486]:
pitches_injury_2015[training_cols].fillna(0).set_index("Name",drop=True).to_csv("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/pitching_data.csv",
                     sep='|')
pitches_injury_2019[training_cols].fillna(0).set_index("Name", drop=True).to_csv("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/pitching_holdout_data.csv",
                                                                                sep='|')
pitches_injury_2020[training_cols].fillna(0).set_index("Name", drop=True).to_csv("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/pitching_holdout_2020_data.csv",
                                                                                sep='|')

In [487]:
pitches_injury_2015.groupby("injury_lagged").agg({"Age":"mean",
                                           "tj":"mean",
                                           "prior_injury":"mean",
                                           "win_rec":"mean",
                                           "IP":"mean",
                                            "ws_pitcher":"mean",
                                                 "pitches_per_bat":"mean",
                                                 "pitches_per_game":"mean"})

,Age,tj,prior_injury,win_rec,IP,ws_pitcher,pitches_per_bat,pitches_per_game
injury_lagged,,,,,,,,
0,29.405757,0.141133,0.375116,0.463073,80.101021,0.038997,3.875158,45.271916
1,29.047414,0.200431,0.476293,0.515199,93.285345,0.032328,3.868595,52.143339


In [451]:
pitches_injury_2019.groupby("injured").agg({"Age":"mean",
                                           "tj":"mean",
                                           "prior_injury":"mean",
                                           "win_rec":"mean",
                                           "IP":"mean",
                                            "ws_pitcher":"mean",
                                                 "walk_rate":"mean",
                                           "pitches_per_bat":"mean",
                                           "pitches_per_game":"mean"})

,Age,tj,prior_injury,win_rec,IP,ws_pitcher,walk_rate,pitches_per_bat,pitches_per_game
injured,,,,,,,,,
0,27.294492,0.069915,0.322034,0.393254,58.812076,0.040254,0.091178,3.904123,39.880807
1,28.593607,0.178082,0.611872,0.405318,63.068037,0.091324,0.082516,3.893039,47.208188


In [ ]:
pitches_injury_2015.groupby("Season").agg({"injury_lagged":"mean"})